In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
import os



base_dir = "/content/drive/MyDrive/gender_data"

# Parameters

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
EPOCHS_INITIAL = 10
EPOCHS_FINE_TUNE = 5
TOTAL_EPOCHS = EPOCHS_INITIAL + EPOCHS_FINE_TUNE
MODEL_SAVE_PATH = "gender_classifier_mobilenetv2.h5"


VAL_SPLIT = 0.2  # 80% train, 20% validation
SEED = 42  # for reproducibility
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])
# train set (80%)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    label_mode="binary",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# val set (20%)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    label_mode="binary",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Prefetch
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


Found 3466 files belonging to 2 classes.
Using 2773 files for training.
Found 3466 files belonging to 2 classes.
Using 693 files for validation.


In [ ]:

base_model = MobileNetV2(input_shape=IMG_SIZE + (3,),
                         include_top=False,
                         weights="imagenet")
base_model.trainable = False

# Preprocessing
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Full model
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)

# Compile
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
]


In [ ]:
# Train (Initial Phase)
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=EPOCHS_INITIAL,
                    callbacks=callbacks)


Epoch 1/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7680 - loss: 0.4725

87/87 ━━━━━━━━━━━━━━━━━━━━ 338s 4s/step - accuracy: 0.7688 - loss: 0.4713 - val_accuracy: 0.9134 - val_loss: 0.2304
Epoch 2/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8953 - loss: 0.2604

87/87 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - accuracy: 0.8954 - loss: 0.2603 - val_accuracy: 0.9278 - val_loss: 0.2046
Epoch 3/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9172 - loss: 0.2260

87/87 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.9171 - loss: 0.2261 - val_accuracy: 0.9307 - val_loss: 0.1948
Epoch 4/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9269 - loss: 0.2036

87/87 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9269 - loss: 0.2037 - val_accuracy: 0.9351 - val_loss: 0.1795
Epoch 5/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9240 - loss: 0.1981

87/87 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.9240 - loss: 0.1981 - val_accuracy: 0.9336 - val_loss: 0.1774
Epoch 6/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9141 - loss: 0.2017

87/87 ━━━━━━━━━━━━━━━━━━━━ 195s 2s/step - accuracy: 0.9142 - loss: 0.2018 - val_accuracy: 0.9365 - val_loss: 0.1655
Epoch 7/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.9372 - loss: 0.1724 - val_accuracy: 0.9322 - val_loss: 0.1867
Epoch 8/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9219 - loss: 0.1992

87/87 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.9219 - loss: 0.1991 - val_accuracy: 0.9394 - val_loss: 0.1648
Epoch 9/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9294 - loss: 0.1849

87/87 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 0.9294 - loss: 0.1850 - val_accuracy: 0.9408 - val_loss: 0.1582
Epoch 10/10
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9332 - loss: 0.1781

87/87 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.9332 - loss: 0.1782 - val_accuracy: 0.9423 - val_loss: 0.1569


In [ ]:
# Fine-tune

base_model.trainable = True

# unreeze top N layers
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

#lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])


history_fine = model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=TOTAL_EPOCHS,
                         initial_epoch=history.epoch[-1] + 1,
                         callbacks=callbacks)


Epoch 11/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 292s 3s/step - accuracy: 0.8065 - loss: 0.4474 - val_accuracy: 0.9250 - val_loss: 0.2068
Epoch 12/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 318s 3s/step - accuracy: 0.8960 - loss: 0.2439 - val_accuracy: 0.9452 - val_loss: 0.1701
Epoch 13/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9251 - loss: 0.2012

87/87 ━━━━━━━━━━━━━━━━━━━━ 263s 3s/step - accuracy: 0.9251 - loss: 0.2011 - val_accuracy: 0.9466 - val_loss: 0.1517
Epoch 14/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9313 - loss: 0.1735

87/87 ━━━━━━━━━━━━━━━━━━━━ 323s 3s/step - accuracy: 0.9313 - loss: 0.1734 - val_accuracy: 0.9538 - val_loss: 0.1435
Epoch 15/15
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9305 - loss: 0.1725

87/87 ━━━━━━━━━━━━━━━━━━━━ 309s 3s/step - accuracy: 0.9305 - loss: 0.1725 - val_accuracy: 0.9567 - val_loss: 0.1401


In [ ]:
# Final Model
model.save(MODEL_SAVE_PATH)
print(f"✅ Model saved to {MODEL_SAVE_PATH}")


✅ Model saved to gender_classifier_mobilenetv2.h5


In [ ]:
# Accuracy/Loss
def plot_history(histories, titles=["Train", "Fine-tune"]):
    acc = []
    val_acc = []
    loss = []
    val_loss = []

    for h in histories:
        acc += h.history['accuracy']
        val_acc += h.history['val_accuracy']
        loss += h.history['loss']
        val_loss += h.history['val_loss']

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Train Acc')
    plt.plot(val_acc, label='Val Acc')
    plt.title("Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Train Loss')
    plt.plot(val_loss, label='Val Loss')
    plt.title("Loss")
    plt.legend()
    plt.show()

plot_history([history, history_fine])


In [ ]:
from google.colab import files
uploaded = files.upload()
